In [165]:
import pandas as pd
import numpy as np
import datetime as dt
from plotnine import ggplot, aes, labs, theme, theme_classic, facet_wrap, element_text
from plotnine import geom_histogram, geom_bar

# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer, SimpleImputer

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, ParameterGrid, KFold, GridSearchCV
import missingno as msno

# Import Data

In [166]:
# X_train = pd.read_csv('0_X_train.csv', index_col='Id')
# X_valid = pd.read_csv('1_X_valid.csv', index_col='Id')
# X_test  = pd.read_csv('2_X_test.csv', index_col='Id')

# y_train = pd.read_csv('0_y_train.csv', index_col='Id')
# y_valid = pd.read_csv('1_y_valid.csv', index_col='Id')
# y_test  = pd.read_csv('2_y_test.csv', index_col='Id')

# #full training set
# X = pd.concat([X_train, X_valid, X_test], axis=0)
# y = pd.concat([y_train, y_valid, y_test], axis=0)

X_pred = pd.read_csv("MLUnige2023_subscriptions_test.csv", index_col="Id")

In [167]:
# y_train = np.array(y_train)
# y_train = y_train.ravel()

# y = np.array(y)
# y = y.ravel()

In [168]:
campaign_ad = pd.read_csv("MLUnige2023_subscriptions_train.csv", index_col="Id")
campaign_ad

,age,job,marital,education,device,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,outcome_old,X1,X2,X3,X4,subscription
Id,,,,,,,,,,,,,,,,,
0,28,freelance,married,grad_school,smartphone,4,2,26.80,3,4,196,failure,0,0,1,0.072803,1
1,48,industrial_worker,married,university,smartphone,30,4,13.05,1,1,79,success,0,0,0,0.075454,1
2,27,teacher,married,university,smartphone,14,7,8.10,3,0,-1,na,0,1,1,0.068110,0
3,44,unemployed,divorced,university,smartphone,13,5,7.10,2,1,369,other,0,0,1,0.091942,0
4,29,manager,single,grad_school,smartphone,26,4,15.90,2,2,143,success,0,0,0,0.085922,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8947,54,industrial_worker,married,university,smartphone,16,7,7.30,1,0,-1,na,1,0,0,0.072803,0
8948,43,industrial_worker,married,university,smartphone,4,2,37.75,2,0,-1,na,0,0,1,0.081456,1
8949,27,manager,single,grad_school,na,5,6,29.00,3,0,-1,na,0,0,1,0.079186,1


# dumify `marital` and `outcome_old`

In [169]:
campaign_ad = pd.get_dummies(campaign_ad, columns=["marital", "outcome_old"])

In [170]:
imputer = SimpleImputer(missing_values = "na", strategy = 'most_frequent')
campaign_ad[['job', 'education', ]] = imputer.fit_transform(campaign_ad[['job', 'education']])

In [171]:
campaign_ad["job"].replace(["entrepreneur", "housekeeper", "industrial_worker", "salesman"], 1, inplace=True)
campaign_ad["job"].replace(["freelance", "manager", "na"], 2, inplace=True)
campaign_ad["job"].replace(["teacher", "technology"], 3, inplace=True)
campaign_ad["job"].replace("retired", 4, inplace=True)
campaign_ad["job"].replace("student", 5, inplace=True)
campaign_ad["job"].replace("unemployed", 6, inplace=True)


# `Device` imputation

In [172]:
# This is the data set that we need to predict and fuse with actual Training data for future imputation of `education`` and `job`
X_tr_dev = campaign_ad.drop(columns=['job', 'education'])
X_tr_imp_dev = X_tr_dev[X_tr_dev['device'] == 'na'].drop(columns = 'device') 
X_tr_imp_dev

,age,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,X1,X2,X3,X4,subscription,marital_divorced,marital_married,marital_single,outcome_old_failure,outcome_old_na,outcome_old_other,outcome_old_success
Id,,,,,,,,,,,,,,,,,,,
5,48,19,6,6.30,2,0,-1,0,0,0,0.078696,0,0,1,0,0,1,0,0
6,31,7,5,24.10,2,0,-1,0,0,1,0.075518,0,0,0,1,0,1,0,0
7,55,17,6,7.00,2,0,-1,0,0,0,0.072903,0,0,1,0,0,1,0,0
22,43,3,6,11.30,7,0,-1,0,0,1,0.073030,0,1,0,0,0,1,0,0
25,42,19,5,4.85,2,0,-1,0,0,0,0.206735,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8922,46,20,6,0.00,3,0,-1,0,0,1,0.092005,0,0,1,0,0,1,0,0
8928,37,6,6,18.55,2,0,-1,1,0,0,0.086494,1,0,1,0,0,1,0,0
8938,36,23,5,3.80,2,0,-1,0,0,1,0.079068,0,0,0,1,0,1,0,0


In [173]:
X_tr_dev = X_tr_dev[X_tr_dev['device'] != 'na'] # The data set that does not have NAs
y_tr_dev = X_tr_dev[['device']] 
X_tr_dev = X_tr_dev.drop(columns = 'device')
X_tr_dev

,age,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,X1,X2,X3,X4,subscription,marital_divorced,marital_married,marital_single,outcome_old_failure,outcome_old_na,outcome_old_other,outcome_old_success
Id,,,,,,,,,,,,,,,,,,,
0,28,4,2,26.80,3,4,196,0,0,1,0.072803,1,0,1,0,1,0,0,0
1,48,30,4,13.05,1,1,79,0,0,0,0.075454,1,0,1,0,0,0,0,1
2,27,14,7,8.10,3,0,-1,0,1,1,0.068110,0,0,1,0,0,1,0,0
3,44,13,5,7.10,2,1,369,0,0,1,0.091942,0,1,0,0,0,0,1,0
4,29,26,4,15.90,2,2,143,0,0,0,0.085922,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8946,32,2,8,23.80,1,2,286,0,0,0,0.148194,0,0,0,1,1,0,0,0
8947,54,16,7,7.30,1,0,-1,1,0,0,0.072803,0,0,1,0,0,1,0,0
8948,43,4,2,37.75,2,0,-1,0,0,1,0.081456,1,0,1,0,0,1,0,0


In [174]:
# Transform categorical features into ordinal integers
enc = OrdinalEncoder(categories = [['smartphone', 'desktop']]) # smartphone = 0, desktop = 1
y_tr_dev = enc.fit_transform(y_tr_dev)
y_tr_dev

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [175]:
y_tr_dev = np.array(y_tr_dev)
y_tr_dev = y_tr_dev.ravel()

In [176]:
rfc_dev = RandomForestClassifier(n_estimators = 100, random_state = 59, n_jobs = -2)
rfc_dev.fit(X_tr_dev, y_tr_dev) # Fit on training, device
y_tr_dev_pred = rfc_dev.predict(X_tr_dev) # Predict on training, device
accuracy_score(y_true = y_tr_dev, y_pred = y_tr_dev_pred) # Compare accuracy from training prediction to true values
#np.array(y_tr_dev_pred.shape)

0.999855114459577

In [177]:
y_tr_imp_dev = rfc_dev.predict(X_tr_imp_dev)
campaign_ad.loc[campaign_ad['device'] == 'smartphone', 'device'] = 0 # smartphone is considered 0 for train
campaign_ad.loc[campaign_ad['device'] == 'desktop', 'device'] = 1 # desktop is considered 1 for train
campaign_ad.loc[X_tr_imp_dev.index, 'device'] = y_tr_imp_dev.astype(int)

# Create Dummies

In [178]:
enc = OrdinalEncoder(categories = [['high_school', 'university', 'grad_school']])
campaign_ad['education'] = enc.fit_transform(campaign_ad[['education']])
campaign_ad.head()

,age,job,education,device,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,...,X3,X4,subscription,marital_divorced,marital_married,marital_single,outcome_old_failure,outcome_old_na,outcome_old_other,outcome_old_success
Id,,,,,,,,,,,,,,,,,,,,,
0,28,2,2.0,0,4,2,26.80,3,4,196,...,1,0.072803,1,0,1,0,1,0,0,0
1,48,1,1.0,0,30,4,13.05,1,1,79,...,0,0.075454,1,0,1,0,0,0,0,1
2,27,3,1.0,0,14,7,8.10,3,0,-1,...,1,0.068110,0,0,1,0,0,1,0,0
3,44,6,1.0,0,13,5,7.10,2,1,369,...,1,0.091942,0,1,0,0,0,0,1,0
4,29,2,2.0,0,26,4,15.90,2,2,143,...,0,0.085922,1,0,0,1,0,0,0,1


In [179]:
campaign_ad = pd.get_dummies(campaign_ad, columns=["job", "education"])
campaign_ad

,age,device,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,X1,X2,...,outcome_old_success,job_1,job_2,job_3,job_4,job_5,job_6,education_0.0,education_1.0,education_2.0
Id,,,,,,,,,,,,,,,,,,,,,
0,28,0,4,2,26.80,3,4,196,0,0,...,0,0,1,0,0,0,0,0,0,1
1,48,0,30,4,13.05,1,1,79,0,0,...,1,1,0,0,0,0,0,0,1,0
2,27,0,14,7,8.10,3,0,-1,0,1,...,0,0,0,1,0,0,0,0,1,0
3,44,0,13,5,7.10,2,1,369,0,0,...,0,0,0,0,0,0,1,0,1,0
4,29,0,26,4,15.90,2,2,143,0,0,...,1,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8947,54,0,16,7,7.30,1,0,-1,1,0,...,0,1,0,0,0,0,0,0,1,0
8948,43,0,4,2,37.75,2,0,-1,0,0,...,0,1,0,0,0,0,0,0,1,0
8949,27,0,5,6,29.00,3,0,-1,0,0,...,0,0,1,0,0,0,0,0,0,1


# Make Partitions

In [180]:
X = campaign_ad.drop(columns = 'subscription')
y = campaign_ad['subscription']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 12) # Test split
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 46) # Validation split

In [181]:
y_train = np.array(y_train)
y_train = y_train.ravel()

y = np.array(y)
y = y.ravel()

# Train Best Model

In [182]:
boost_hyperparam_grid={'max_depth':[1, 5, 9, 11], 
                       "n_estimators":[200, 1000, 1250], 
                       "learning_rate":[0.001, 0.01, 0.1]}

boost = GradientBoostingClassifier(max_features=None, 
                             warm_start=False, random_state=1)

best_boost_score=0.5

for bg in ParameterGrid(boost_hyperparam_grid):
    boost.set_params(**bg)
    boost.fit(X_train,y_train)
    # save if best
    if boost.score(X_train, y_train) > best_boost_score:
        best_boost_score = boost.score(X_train, y_train)
        best_boost_params = bg

print(f"OOB: %0.5f" % best_boost_score)
print("Best parameters:", best_boost_params)

In [163]:
boost = GradientBoostingClassifier(max_depth = 12, 
                                   n_estimators = 1000,
                                   learning_rate = 0.001,
                                   random_state=1)

In [164]:
#Print the results
print("Boosted tree Best Model")
print(" ")

#fit and predict on partitioned data set
np.random.seed=(45)
boost.fit(X_train,y_train)
print("Training accuracy:", round(boost.score(X_train, y_train),4))
print("Validation accuracy:", round(boost.score(X_valid, y_valid),4))
print("Test accuracy:", round(boost.score(X_test, y_test),4))
print("X accuracy for Partially Trained Model:", round(boost.score(X, y),4))

#fit and predict on all data
np.random.seed=(45)
boost.fit(X,y)
print("X accuracy on Fully Trained Model:", round(boost.score(X, y),4))

Boosted tree Best Model
 
Training accuracy: 0.9805
Validation accuracy: 0.8012
Test accuracy: 0.8243
X accuracy for Partially Trained Model: 0.9302
X accuracy on Fully Trained Model: 0.9684
